# A TOOL FOR THE RISK ASSESSMENT

In this Jupyter Notebook, we will implement the necessary formulas for risk assessment to determine whether there is a potential risk based on seven parameters defined by the user:
- **Concentration of the contaminant** (in mg/l)
- **Weight of the person of interest** (in kg)
- **Average time** (in years)
- **Exposure time** (in years)
- **Specific contaminant of concern**
- **Type of affected** (adult, children or labor adult)
- **Scenario of exposure** 

This notebook is fed by an Excel sheet that provides data on several contaminants, including reference doses and toxicity factors. These are categorized by the route of exposure (oral, inhalation, and dermal contact), the type of effect (systemic or carcinogenic), and the exposure duration (chronic or subchronic).

The sources used for the creation of the database are:
- [BOE](https://www.boe.es/buscar/pdf/2005/BOE-A-2005-895-consolidado.pdf) (in Spanish)
- [RAIS](https://rais.ornl.gov/)
- [IRIS](https://cfpub.epa.gov/ncea/iris/search/index.Cfm)
- [EPA](https://www.epa.gov/expobox/about-exposure-factors-handbook)


**Note:** The Excel sheet ```db_contaminants.xlsx``` must be available in the same project folder as the ```risk_assessment_tool.ipynb```.

## Exposure definition 
<br>

\begin{array}{|l|l|}
\hline
\text{$\quad\quad\quad$ \textbf{Scenarios}} & \text{$\quad\quad\quad$ \textbf{Subscenarios}} \\
\hline
                                    & \text{} \\
                                    & \text{E01 Water Intake} \\
\text{E0 Water or Food Ingestion}   & \text{E02 Vegetables Intake} \\
                                    & \text{E03 Meat and Dairy Intake} \\
                                    & \text{}\\
\hline
                                    & \text{}\\
\text{E1 Agricultural}              & \text{E11 Outdoor Irrigation} \\
                                    & \text{E12 Greenhouse Irrigation} \\
                                    & \text{}\\
\hline
                                    & \text{}\\
\text{E2 Industrial}                & \text{E21 Personal Cleaning} \\
                                    & \text{E22 Industrial Cleaning} \\
                                    & \text{}\\
\hline
                                    & \text{}\\
\text{E3 Domestic}                  & \text{E31 Domestic Hygiene} \\
                                    & \text{E32 Private Gardens} \\
                                    & \text{}\\
\hline
                                    & \text{}\\
\text{E4 Urban}                     & \text{E41 Street Cleaning} \\
                                    & \text{E42 Urban Cleaning} \\
                                    & \text{}\\
\hline
                                    & \text{}\\
\text{E5 Leisure}                   & \text{E51 Swimming Pool Bath} \\
                                    & \text{}\\
\hline
\end{array}

<br>

**Disclaimer** This Jupyter Notebook currently focuses solely on oral exposure, covering the following four subscenarios:
- E01: Water Intake
- E21: Personal Cleaning
- E31: Domestic Hygiene
- E51: Swimming Pool Bath

<br>

## Formulas used to define risk

### Average Daily Dose
Average Daily Dose (ADD) is generally expressed as mass of contaminant per unit body weight over time, and is defined as follows.

$$ ADD = \bar{C}_w\left[{IR \over BW}\right] {ED \times EF \over AT} $$

Where, 

$\bar{C}_w$ maximum concentration in the water (mg/l)\
$IR$ the daily water intake (l/d)\
$BW$ body weight (Kg)\
$ED$ exposure duration (years)\
$EF$ exposure frequency (days/year)\
$AT$ averaging time (years)\

### Target Hazard Quotient
The target hazard quotient (THQ) is defined as the ratio of exposure to the toxic element and the reference dose which is the highest level at which no adverse health effects are expected. If the THQ is <1 then adverse health effects are not expected. If, however, the THQ is >1 then there is a possibility that adverse health effects could be experienced. THQ is defined as follows.
$$ THQ = {ADD \over RfD} $$

Where,

$RfD$ reference dose (mg/Kg day)

### Target Risk
The target risk (TR) is used to assess the potential risk associated with exposure to toxic agents throughout the lifetime exposure period. Instead of an reference dose (RfD), as is used for the determination of THQ, an oral slope factor (SF) is used. This factor determines the probability of excess health risk over the lifetime of the exposed individual. If TR >$10^{-5}$, there is the potential for adverse carcinogenic health effect. TR is defined as follows. 
$$ TR = ADD \times SF $$

Where,

$SF$ oral slope factor (kg day/mg)

References:
- Antoine, J. M., Fung, L. A. H., & Grant, C. N. (2017): "Assessment of the potential health risks associated with the aluminium, arsenic, cadmium and lead content in selected fruits and vegetables grown in Jamaica". Toxicology reports, 4, 181-187.

<br>

### Coding the equations for the risk assessment

#### Importing the needed libraries

In [ ]:
#-- Check and install required packages if not already installed --#
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed --#
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")

#-- Import the rest of libraries --#
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
import pandas as pd
import numpy as np
import os

from IPython.display import display, clear_output

import warnings
warnings.filterwarnings("ignore")

#### The dataframe, the scenarios and the affected

In [ ]:
#------------------------------------------#
# Importing the dataframe                  #
#------------------------------------------#
BD_contaminants = pd.read_excel("db_contaminants.xlsx")

#------------------------------------------#
# Defining the contaminants                #
#------------------------------------------#
table_of_contaminants = pd.DataFrame({'Contaminants': [], 
                                      'CAS': [], 
                                      'RfDoc [mg/Kg day]': [], 
                                      'SubRfDo [mg/Kg day]': [], 
                                      'SFo [Kg day/mg]': [], 
                                      'RfC [mg/m3]': [], 
                                      'RfDi [mg/Kg day]': [], 
                                      'RfCs [mg/m3]': [], 
                                      'SubRfDi [mg/Kg day]': [], 
                                      'IUR [ug/m3]': [], 
                                      'SFI [Kg day/mg]': []
                                     })
table_of_contaminants['Contaminants'] = BD_contaminants.iloc[5:, 0].reset_index(drop=True)
table_of_contaminants['CAS'] = BD_contaminants.iloc[5:, 1].reset_index(drop=True)
table_of_contaminants['RfDoc [mg/Kg day]'] = BD_contaminants.iloc[5:, 2].reset_index(drop=True)
table_of_contaminants['SubRfDo [mg/Kg day]'] = BD_contaminants.iloc[5:, 3].reset_index(drop=True)
table_of_contaminants['SFo [Kg day/mg]'] = BD_contaminants.iloc[5:, 4].reset_index(drop=True)
table_of_contaminants['RfC [mg/m3]'] = BD_contaminants.iloc[5:, 5].reset_index(drop=True)
table_of_contaminants['RfDi [mg/Kg day]'] = BD_contaminants.iloc[5:, 6].reset_index(drop=True)
table_of_contaminants['RfCs [mg/m3]'] = BD_contaminants.iloc[5:, 7].reset_index(drop=True)
table_of_contaminants['SubRfDi [mg/Kg day]'] = BD_contaminants.iloc[5:, 8].reset_index(drop=True)
table_of_contaminants['IUR [ug/m3]'] = BD_contaminants.iloc[5:, 9].reset_index(drop=True)
table_of_contaminants['SFI [Kg day/mg]'] = BD_contaminants.iloc[5:, 10].reset_index(drop=True)

#------------------------------------------#
# Defining the users                       #
#------------------------------------------#
list_of_users = ['Resident Adult', 'Children', 'Labor Adult']

#------------------------------------------#
# Defining the scenarios                   #
#------------------------------------------#
dict_of_scenarios = {'E0 Water and Food Ingestion': ['E1 Water or Food Ingestion', 'E02 Vegetables Intake', 'E03 Meat and Dairy Intake'], 
                     'E1 Agricultural': ['E11 Outdoor Irrigation', 'E12 Greenhouse Irrigation'], 
                     'E2 Industrial': ['E21 Personal Cleaning', 'E22 Industrial Cleaning'], 
                     'E3 Domestic': ['E31 Domestic Hygiene', 'E32 Private Gardens'], 
                     'E4 Urban': ['E41 Street Cleaning', 'E42 Urban Cleaning'], 
                     'E5 Leisure': ['E51 Swimming Pool Bath']}

#------------------------------------------#
# Defining the daily water intake (IR)     #
#------------------------------------------#
df_IR = pd.DataFrame(columns=['Scenarios'] + list_of_users)

df_IR.iloc[:,0] = ['E01: Water Intake', 'E21: Personal Cleaning', 'E31: Domestic Hygiene', 'E51: Swimming Pool Bath']
df_IR.iloc[:,1] = [2, 0, 0.1, 0.26]
df_IR.iloc[:,2] = [1, 0, 0.05, 0.13]
df_IR.iloc[:,3] = [0, 0.04, 0, 0]

df_IR.head()


####  The formulation


In [ ]:
#------------------------------------------#
# Functions defining risk                  #
#------------------------------------------#
def average_daily_dose(Cw: float, IR: float, BW: int, ED: float, EF: float, AT: int) -> float:
    """Function that calculates the average daily dose (ADD)

    Parameters
    ----------
    Cw : float
        maximum concentration in water
    IR : float
        daily water intake
    BW : int
        body weight
    ED : float
        exposure duration
    EF : float
        exposure frequency
    AT : int
        average time

    Returns
    -------
    float
        average daily dose
    """
    ADD = Cw*(IR/BW)*((ED*EF)/(AT*365))
    return ADD

def target_hazard_quotient(ADD: float, RfDo: float) -> float:
    """Function that calculates the hazard quotient (THQ)

    Parameters
    ----------
    ADD : float
        average daily dose 
    RfDo : float
        reference dose

    Returns
    -------
    float
        the hazard quotient
    """
       
    THQ = ADD/RfDo
    return THQ

def target_risk(ADD: float, SF: float) -> float:
    """Function that calculates the target risk 

    Parameters
    ----------
    ADD : float
        average daily dose
    SF : float
        oral slope factor

    Returns
    -------
    float
        target risk
    """
      
    TR = ADD*SF
    return TR

#------------------------------------------#
# Functions defining the widgets           #
#------------------------------------------#
def create_widget(description: str, style=None, width=None):
    """
    Function for the generation of widget 

    Parameters:
        - description (str): description of the widget
    Return:
        - widget_value: returns the value of the widget
    """ 
    widget_value = widgets.Text(value='', description=description, 
                                style={'description_width': 'initial'}, width = '500px')
    return widget_value


def dropdown_widget(description: str, options: str, value: str):
    """
    Function for the generation of the dropdown widget.
    
    Parameters:
        - description (str): description of the widget
        - options (str): the values to choose in the widget
        - value (str): the value to initialize the widget
    Return:
        - widget_value: returns the value of the widget
    """ 
    widget_value = widgets.Dropdown(description=description, options=options, 
                                    value = value, style = {'description_width': 'initial'},
                                    width = '400px', disabled=False)
    return widget_value
 
def update_numeric_value(widget, variable):
    """Observes changes in the widget's value and updates the variable accordingly.

    When a user enters or modifies text in the widget, the 'update_value' function 
    will be automatically called, allowing you to respond to and handle the changes 
    in the widget's value. 

    Parameters
    ----------
    widget : ipywidgets.Widget
        The widget object that will be observed for changes in its value
    variable : Any
        A global variable to be updated based on the widget's input

    Returns
    -------
    widget : ipywidgets.Widget
        The original widget, with an observer attached to monitor changes

    Notes
    -----
    If the new value in the widget is blank, 'variable' will be set to None. If the 
    input is invalid, an error message will be displayed
    """
    def update_value(change):
        global variable
        new_value = change.new.strip()
        if new_value == '':
            variable = None
        else:
            try:
                variable = float(new_value)
            except ValueError:
                print(f"Invalid input for '{widget.description}', please enter a valid value.")

    widget.observe(update_value, names='value')
    return widget


def CW_input(Cw):
    """
    Concentration found (mg/l)
    """
    cw_wid = create_widget(description='Concentration (mg/l):',
                           style={'description_width': 'initial'}, width='500px')  
    cw_wid = update_numeric_value(cw_wid, Cw)
    return cw_wid

def BW_input(BW):
    """
    Body Weight (kg)
    """
    bw_wid = create_widget(description='Weight (kg):', style={'description_width': 'initial'}, 
                           width='500px')
    bw_wid = update_numeric_value(bw_wid, BW)
    return bw_wid

def AT_input(AT):
    """
    Average time (years)
    """
    at_wid = create_widget(description='Average time (years):', style={'description_width': 'initial'}, 
                           width='500px')
    at_wid = update_numeric_value(at_wid, AT)
    return at_wid

def ED_input(ED):
    """
    Exposure time (years)
    """
    ed_wid = create_widget(description='Exposure time (years):', style={'description_width': 'initial'}, 
                           width='500px')
    ed_wid = update_numeric_value(ed_wid, ED)
    return ed_wid

def contaminant_input():
    """
    Widget for the contaminant
    """
    contaminant_wid = dropdown_widget(description = 'Choose the contaminant:', 
                    options = table_of_contaminants['Contaminants'], value = 'Benceno')
    return contaminant_wid

def affected_input():
    """
    Widget for the affected
    """
    affected_wid = dropdown_widget(description = 'Choose the affected:', 
                    options = list_of_users, value = 'Resident Adult')
    return affected_wid

def scenario_input():
    """
    Widget for the scenarios
    """
    scenario_wid = dropdown_widget(description = 'Choose the scenario:', 
                    options = df_IR.iloc[:,0], value = 'E01: Water Intake')
    return scenario_wid


#------------------------------------------#
# The outcome                              #
#------------------------------------------#
output = widgets.Output()

#-- Button to trigger the calculation
button = widgets.Button(button_style='info', description="Calculate Risk",
                            tooltip='Click me', icon='check')

def on_button_clicked(button):
    """Retrieve input values from widgets and perform risk calculations.

    This function gathers user inputs, including contaminant concentration, 
    body weight, average time, exposure duration, and more, to compute the 
    average daily dose (ADD), target hazard quotient (THQ), and target risk (TR).
    Based on the calculated TR value, the function provides a risk assessment.

    Parameters
    ----------
    button : widgets.Button
        A button widget that triggers the calculation when clicked
    """
    Cw = float(cw_wid.value) 
    BW = float(bw_wid.value) 
    AT = float(at_wid.value) 
    ED = float(ed_wid.value) 
    EF_E51 = 36  
    contaminant_name = contaminant_wid.value
    affected_one = affected_wid.value
    
    if affected_one == "Labor Adult":
        EF = 250
    else:
        EF = 350  
    
    scenario = scenario_wid.value
    IR = df_IR.loc[df_IR['Scenarios'] == scenario, affected_one].values[0] 
    RfDo = table_of_contaminants.loc[table_of_contaminants['Contaminants'] == contaminant_name, 
                                     'RfDoc [mg/Kg day]']
    SF = table_of_contaminants.loc[table_of_contaminants['Contaminants'] == contaminant_name, 
                                   'SFo [Kg day/mg]']
    
    ADD = average_daily_dose(Cw, IR, BW, ED, EF, AT)
    THQ = target_hazard_quotient(ADD, RfDo)
    TR = target_risk(ADD, SF)

    with output:
        output.clear_output(wait=True)
        print(f"The average daily dose (ADD) value is {ADD:e}\n")
        print(f"The target hazard quotient (THQ) value is {float(THQ.iloc[0]):e}\n")
        print(f"The target risk (TR) value is then {float(TR.iloc[0]):e} so...\n\n")

        if float(TR.iloc[0]) < 10E-5:
            print("No significant risk detected.\n\n\n")
        else:
            print("RISK DETECTED!\n\n\n")

#-- Attaching the function to the button's click event
button.on_click(on_button_clicked)

#-- Displaying the widgets and button
cw_wid = CW_input(Cw=0)  
bw_wid = BW_input(BW=0)  
at_wid = AT_input(AT=0)  
ed_wid = ED_input(ED=0)  

contaminant_wid = contaminant_input()
affected_wid = affected_input()
scenario_wid = scenario_input()

#-- Defining the order of widgets
widgets_list = [cw_wid, bw_wid, at_wid, ed_wid, contaminant_wid, affected_wid, scenario_wid, button]

#-- Displaying the widgets
display(widgets.VBox(widgets_list), output)